# Trabalho de Algoritmos e Estrutura de Dados
## Problema da Mochila

## Importando as dependências

In [2]:
from datetime import datetime
import pandas as pd

## Definindo o problema

In [2]:
# dados = {"importancia": [60, 100, 120, 50],
#          "valor": [10, 20, 30, 50],
#          "selecionado": [0, 0, 0, 0]}
# valor_disponivel = 50

# dados = {"importancia": [10, 21, 50, 51],
#          "valor": [2, 3, 5, 6],
#          "selecionado": [0, 0, 0, 0]}
# valor_disponivel = 7
# itens = pd.DataFrame(dados)

valor_disponivel = 6200000 * 100
itens = pd.read_excel("proposicoes_STI_2023.xlsx", sheet_name="Tratado")
itens = itens.filter(["Ação", "GUT", "Unidade Total"]).rename(columns={"Ação": "acao", "GUT": "importancia",
                                                                       "Unidade Total": "valor_original"})
itens["valor"] = itens.valor_original * 100

itens["importancia_por_valor"] = itens.importancia / itens.valor
itens["selecionado"] = 0

## Algoritmos Exatos
---

### Força Bruta

In [3]:
# from algoritmos_exatos import BruteForceKnapsack

In [4]:
# inicio_processamento = datetime.now()
#
# knapsack = BruteForceKnapsack(capacidade, itens)
# valor, peso = knapsack.max()
# print(f"Valor máximo: {valor} / Peso máximo: {peso / 100}")
#
# fim_processamento = datetime.now()
# print("Tempo de processamento:", fim_processamento - inicio_processamento)

### Programação Dinâmica

In [5]:
# from algoritmos_exatos import DynamicProgrammingKnapsack

In [6]:
# inicio_processamento = datetime.now()
#
# knapsack = DynamicProgrammingKnapsack(capacidade, itens)
# print(f"Valor máximo que obtemos = {knapsack.max()}")
#
# fim_processamento = datetime.now()
# print("Tempo de processamento:", fim_processamento - inicio_processamento)

### Branch and Bound

In [7]:
from algoritmos_exatos import BranchAndBoundKnapsack

In [8]:
inicio_processamento = datetime.now()

knapsack = BranchAndBoundKnapsack(capacidade, itens)
valor, peso = knapsack.max()
print(f"Retorno máximo: {valor} / Peso máximo: {peso / 100}")

fim_processamento = datetime.now()
print("Tempo de processamento:", fim_processamento - inicio_processamento)

Retorno máximo: 432471472.7784533 / Peso máximo: 6153187.28
Tempo de processamento: 0:00:00.039987


## Algoritmos Aproximados
---

### Algoritmo Guloso

In [9]:
from algoritmos_aproximados import GreedyKnapsack

In [10]:
inicio_processamento = datetime.now()

knapsack = GreedyKnapsack(capacidade, itens)
valor, peso = knapsack.max()
print(f"Retorno máximo: {valor} / Valor máximo: {peso / 100}")

fim_processamento = datetime.now()
print("Tempo de processamento:", fim_processamento - inicio_processamento)

Retorno máximo: 2199 / Valor máximo: 5713187.28
Tempo de processamento: 0:00:00.004000


In [11]:
inicio_processamento = datetime.now()

knapsack = GreedyKnapsack(capacidade, itens, fractional=True)
valor, peso = knapsack.max()
print(f"Retorno máximo: {valor} / Valor máximo: {peso / 100:0.2f}")

fim_processamento = datetime.now()
print("Tempo de processamento:", fim_processamento - inicio_processamento)

Retorno máximo: 2215.2546945714284 / Valor máximo: 5784403.06
Tempo de processamento: 0:00:00.003997


In [7]:
serie = pd.Series([0, 1, 2, 3, 4])
serie.iloc([2, 4])

TypeError: unhashable type: 'list'